In [1]:
from dataloader import Load_Dataset 
from models import models
from predict import predict
from models.models import Attention_layer,Attention_model
from utils import utils
from train import train
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence



from gensim.models import FastText
from gensim.test.utils import datapath
# from fastText import train_supervised

from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib
import numpy as np
import itertools
import scipy
import matplotlib.pyplot as plt
# import fastText
# import pagexml
# from pagexmltools.process import page_region_with_ordered_textlines
import numpy as np
import os
import re
import nltk
import sys
# import pagexmltools
%matplotlib inline
import statistics 

from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from scipy.spatial import distance_matrix
import seaborn as sns; sns.set()
from torch import nn
import glob
import os
import csv
import sys
import pickle
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
# from nlp_classification.nlp_classification.pre_processor import gaussianize
#from keras.utils import to_categorical
from sklearn.preprocessing import normalize
import warnings
# warnings.filterwarnings('ignore')

In [ ]:
class Config_dataloader(object):    
    '''
    Load_dataset
    '''

    '''Data type'''
    benchmark_data =True
    Ngrams =False
    kmeans =False
    '''parameters'''
    N_GRAMS =5
    N_CLUSTERS =15
    dump_fasttext_file = '/home/ankit/experiment_framework/train_fasttext_data/train_fast_text.txt'
    


In [5]:
'''Load dataset dict'''
config = Config_dataloader()

path_to_mergedxml ='/home/ankit/Thesis/data/azp-booking-confirmation/merged_file_list.txt'
data_dict_   = Load_Dataset.load_dataset(config, path_to_mergedxml)
word_to_ix =data_dict_['word_to_ix']
tag_to_ix =data_dict_['tag_to_ix']

Total unique words in dataset 18532
Total unique Tags in dataset 6
Fasttext training data at -> /home/ankit/experiment_framework/train_fasttext_data/train_fast_text.txt


In [6]:
'''Softmax'''
data = data_dict_['data']

In [25]:
'''Extra step for attention'''
data_for_attention = utils.return_padded_data_dict(data_dict_)
word_to_ix =data_for_attention['word_to_ix']
tag_to_ix =data_for_attention['tag_to_ix']
data = data_for_attention['data']

In [30]:
class Config_SOFTMAX(object):
    '''LSTM'''
    embedding_dim = 100
    hidden_dim = 10
    num_layer=1
    embedding_model = utils.load_embedding_model('model_complete.bin')
    use_fasttext= True    # if false model will inititalize embedding layer
    vocab_size = len(word_to_ix)   #fiix
    tagset_size = len(tag_to_ix)
    max_length = len(data[0][0])    
    
    '''Training'''
    num_epochs =50
    learning_rate =0.001
    
    '''LOSS'''
    use_crf =False    #either of them has to be false/True
    use_softmax= True    #when using attention only use softmax
    
    '''Paths'''
    path_to_saved_model = '/home/ankit/experiment_framework/saved_models/'
    
    
    '''CNN specific'''    # If using CNN. use_softmax= True
    n_class = 6

In [7]:
# model = Attention_model(Config_SOFTMAX)

In [8]:
'''Init model'''
config_soft = Config_SOFTMAX()
model = models.Attention_model(config_soft)
#loss_function = nn.NLLLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.01)

In [9]:
# data = data_dict_['data']    #-> tuple

[['introduisez',
  'le',
  'code',
  'barres',
  'dans',
  'les',
  'machines',
  'check-in',
  'insert',
  'the',
  'barcode',
  'into',
  'the',
  'check-in',
  'machines',
  'votre',
  'billet',
  'électronique',
  'et',
  "l'itinéraire",
  'your',
  'electronic',
  'ticket',
  'and',
  'itinerary',
  'enregistrement',
  'necessaire',
  'UNK',
  'need',
  'to',
  'check',
  'in',
  'informations',
  'du',
  'passager',
  'passenger',
  'data',
  'nom',
  'non',
  'transferable',
  'name',
  'nottransferable',
  'papier',
  "d'identité",
  'UNK',
  'identitv',
  'document',
  'code',
  'de',
  'réservation',
  'booking',
  'code',
  'numéro',
  'UNK',
  'number',
  'information',
  'des',
  'vols',
  'flight',
  'data',
  'origine/destination',
  'vol',
  'départ',
  'arrivée',
  'non',
  'valide',
  'classe',
  '-',
  'tarif',
  'franchise',
  'status',
  'origin/destination',
  'flight',
  'departure',
  'arrivai',
  'not',
  'valid',
  'class',
  '-',
  'fare',
  'baggage',
  'sta

In [10]:
'''Training'''
train.train_loop(Config_SOFTMAX,model,data,tag_to_ix,use_attention=True)

/home/ankit/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Running epoch 1
Running epoch 2
Running epoch 3
Running epoch 4
Running epoch 5
Running epoch 6
Running epoch 7
Running epoch 8
Running epoch 9
Running epoch 10
Running epoch 11
Running epoch 12
Running epoch 13
Running epoch 14
Running epoch 15
Running epoch 16
Running epoch 17
Running epoch 18
Running epoch 19
Running epoch 20
Running epoch 21
Running epoch 22
Running epoch 23
Running epoch 24
Running epoch 25
Running epoch 26
Running epoch 27
Running epoch 28
Running epoch 29
Running epoch 30
Running epoch 31
Running epoch 32
Running epoch 33
Running epoch 34
Running epoch 35
Running epoch 36
Running epoch 37
Running epoch 38
Running epoch 39
Running epoch 40
Running epoch 41
Running epoch 42
Running epoch 43
Running epoch 44
Running epoch 45
Running epoch 46
Running epoch 47
Running epoch 48
Running epoch 49
Running epoch 50
Saving models at-> /home/ankit/experiment_framework/saved_models/


In [11]:
# Init model with same config and give it a path to one of the checkpoints
'''Predict usage'''
'''INIT model resume'''
#model = models.LSTMTagger_softmax(Config_SOFTMAX)


p,r,f = predict.predict_for_all_epochs(model,Config_SOFTMAX,data,tag_to_ix)   #it will pick model from path in config

/home/ankit/anaconda3/envs/python3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


KeyboardInterrupt: 

In [19]:
# plt.plot(p)
len(data)

4005

In [31]:
predict.predict(model,config_soft,49,data,tag_to_ix)

RuntimeError: shape '[-1, 933]' is invalid for input of size 8575

In [ ]:
plt.plot(r)

In [ ]:
plt.plot(f)

'/home/ankit/Thesis/data/azp-booking-confirmation/merged_file_list.txt'

In [2]:
'''temporary for softmax '''
class Config_dataloader(object):    
    '''
    Load_dataset
    '''

    '''Data type'''
    benchmark_data =True
    Ngrams =False
    kmeans =False
    '''parameters'''
    N_GRAMS =5
    N_CLUSTERS =15
    dump_fasttext_file = '/home/ankit/experiment_framework/train_fasttext_data/train_fast_text.txt'

In [3]:
'''Load dataset dict'''
config = Config_dataloader()

path_to_mergedxml ='/home/ankit/Thesis/data/azp-booking-confirmation/train_xmls.txt'
data_dict_   = Load_Dataset.load_dataset(config, path_to_mergedxml)
word_to_ix =data_dict_['word_to_ix']
tag_to_ix =data_dict_['tag_to_ix']

Total unique words in dataset 16781
Total unique Tags in dataset 6
Fasttext training data at -> /home/ankit/experiment_framework/train_fasttext_data/train_fast_text.txt


In [4]:
data = data_dict_['data']

In [5]:
config = Config_dataloader()

path_to_mergedxml ='/home/ankit/Thesis/data/azp-booking-confirmation/test_xmls.txt'
test_data_dict_   = Load_Dataset.load_dataset(config, path_to_mergedxml)
word_to_ix =test_data_dict_['word_to_ix']
tag_to_ix =test_data_dict_['tag_to_ix']
test_data = test_data_dict_['data'] 

Total unique words in dataset 5423
Total unique Tags in dataset 6
Fasttext training data at -> /home/ankit/experiment_framework/train_fasttext_data/train_fast_text.txt


In [9]:
class Config_SOFTMAX(object):
    '''LSTM'''
    embedding_dim = 100
    hidden_dim = 10
    num_layer=1
    embedding_model = utils.load_embedding_model('model_complete.bin')
    use_fasttext= True    # if false model will inititalize embedding layer
    vocab_size = len(word_to_ix)   #fiix
    tagset_size = len(tag_to_ix)
    max_length = len(data[0][0])    
    
    '''Training'''
    num_epochs =50
    learning_rate =0.001
    
    '''LOSS'''
    use_crf =False    #either of them has to be false/True
    use_softmax= True    #when using attention only use softmax
    
    '''Paths'''
    path_to_saved_model = '/home/ankit/experiment_framework/saved_models/softmax/'
    
    
    '''CNN specific'''    # If using CNN. use_softmax= True
    n_class = 6

In [10]:
'''Init model'''
config_soft = Config_SOFTMAX()
model = models.LSTMTagger_softmax(config_soft)
#loss_function = nn.NLLLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
train.train_loop(Config_SOFTMAX,model,data,tag_to_ix,use_attention=False)

Running epoch 1


/home/ankit/anaconda3/envs/python3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Prercision, recall, F1score-> (0.0001922306768121746, 0.007079646017699115, 0.00037429819089207733, None)
Prercision, recall, F1score-> (0.00011837121212121212, 0.0029498525073746312, 0.00022760896779333104, None)
Prercision, recall, F1score-> (0.00010859728506787331, 0.0017699115044247787, 0.0002046384720327422, None)
Prercision, recall, F1score-> (0.00016460905349794237, 0.0017699115044247787, 0.0003012048192771084, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)


Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, 

Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, recall, F1score-> (0.0, 0.0, 0.0, None)
Prercision, 

In [ ]:
p_,r_,f_ = predict.predict_for_all_epochs(model,Config_SOFTMAX,data,tag_to_ix)   #it will pick model from path in config

In [ ]:
plt.plot(p_)

In [ ]:
plt.plot(r_)

In [ ]:
plt.plot(f_)

In [ ]:
'''temporary for CRF '''
class Config_dataloader(object):    
    '''
    Load_dataset
    '''

    '''Data type'''
    benchmark_data =False
    Ngrams =False
    kmeans =True
    '''parameters'''
    N_GRAMS =5
    N_CLUSTERS =15
    dump_fasttext_file = '/home/ankit/experiment_framework/train_fasttext_data/train_fast_text.txt'
    
    
'''Load dataset dict'''
config = Config_dataloader()

path_to_mergedxml ='/home/ankit/Thesis/data/azp-booking-confirmation/merged_file_list.txt'
data_dict_   = Load_Dataset.load_dataset(config, path_to_mergedxml)
word_to_ix =data_dict_['word_to_ix']
tag_to_ix =data_dict_['tag_to_ix']

data = data_dict_['data']


class Config_SOFTMAX(object):
    '''LSTM'''
    embedding_dim = 100
    hidden_dim = 10
    num_layer=1
    embedding_model = utils.load_embedding_model('model_complete.bin')
    use_fasttext= True    # if false model will inititalize embedding layer
    vocab_size = len(word_to_ix)   #fiix
    tagset_size = len(tag_to_ix)
    max_length = len(data[0][0])    
    
    '''Training'''
    num_epochs =50
    learning_rate =0.001
    
    '''LOSS'''
    use_crf =True   #either of them has to be false/True
    use_softmax= False    #when using attention only use softmax
    
    '''Paths'''
    path_to_saved_model = '/home/ankit/experiment_framework/saved_models/crf'
    
    
    '''CNN specific'''    # If using CNN. use_softmax= True
    n_class = 6
    
    
    
'''Init model'''
config_soft = Config_SOFTMAX()
model = models.LSTMTagger_CRF(config_soft)

train.train_loop(Config_SOFTMAX,model,data,tag_to_ix,use_attention=False)
p__,r__,f__ = predict.predict_for_all_epochs(model,Config_SOFTMAX,data,tag_to_ix)   #it will pick model from path in config

plt.plot(p__)
plt.plot(r__)
plt.plot(f__)

In [ ]:
 j 